In [32]:
import psycopg2
from psycopg2.extras import execute_values
import os
from dotenv import load_dotenv
from datetime import datetime
import pandas as pd
from tqdm import tqdm

load_dotenv()

True

In [22]:
db_host = os.getenv("DB_HOST")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_pass = os.getenv("DB_PASSWORD")

conn = psycopg2.connect(
            host=db_host,
            database=db_name,
            user=db_user,
            password=db_pass
)

In [15]:
import re
import psycopg2


def normalize_number(phone: str) -> str:
    """Remove spaces, dashes, brackets etc."""
    phone = re.sub(r"[^\d+]", "", phone)
    return phone


def detect_country_code(phone: str, conn) -> str | None:
    """
    Detect country calling code using DB reference table.
    Tries longest match first.
    """
    phone = normalize_number(phone)

    with conn.cursor() as cur:
        # Get all calling codes sorted by length DESC
        cur.execute("""
            SELECT calling_code, iso2
            FROM country_calling_codes
            ORDER BY LENGTH(calling_code) DESC
        """)
        codes = [(row[0], row[1]) for row in cur.fetchall()]

    for code, iso2 in codes:
        if phone.startswith(code):
            return iso2

    return None

In [33]:
def update_whitelist_country_code(conn):
    with conn.cursor() as cur:
        cur.execute("SELECT custom_variable_1 FROM whitelist")
        rows = cur.fetchall()

        for phone in tqdm(rows):
            code = detect_country_code(phone[0], conn)
            cur.execute("""
                UPDATE whitelist
                SET country_code = %s
                WHERE custom_variable_1 = %s
            """, (code, phone))

    conn.commit()

In [34]:
update_whitelist_country_code(conn)

  0%|          | 147/316995 [00:37<22:14:55,  3.96it/s]


KeyboardInterrupt: 